In [3]:
import cv2
import numpy as np
import os
import pickle 

# Initialize the webcam
cap = cv2.VideoCapture(0)  # Access the webcam

# Load the face detection model using Haar Cascades
face_detect = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize a list to store face data
face_data = []

# Prompt the user to enter their name
name = input('Enter your name: ')

# Initialize a counter to capture every 10th frame
i = 0  # Frame counter to take an image every 10 frames

# Main loop for capturing and saving face images
while True:
    ret, frame = cap.read()  # Read a frame from the webcam
    if not ret:
        break  # Break if the frame is not captured properly
    
    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the grayscale frame
    faces = face_detect.detectMultiScale(gray, scaleFactor=1.02, minNeighbors=5)

    # Loop over detected faces
    for (x, y, w, h) in faces:
        # Crop the detected face from the frame
        crop_image = frame[y:y+h, x:x+w, :]
        # Resize the cropped face image to 50x50 pixels
        resized_image = cv2.resize(crop_image, (50, 50))
        
        # Collect 50 face samples, taking one every 10 frames
        if len(face_data) < 50 and i % 10 == 0:
            face_data.append(resized_image)  # Add the resized face image to the list
            # Display the number of collected images on the frame
            cv2.putText(frame, str(len(face_data)), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 255))
            # Draw a rectangle around the detected face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
    
    # Display the frame with the face rectangles and text
    cv2.imshow('frame', frame)
    i += 1  # Increment the frame counter
    
    # Check if we have collected enough face images
    if len(face_data) == 50:
        break

# Release the webcam and close all OpenCV windows
cap.release() 
cv2.destroyAllWindows()  

# Convert the list of collected faces to a NumPy array
face_data = np.array(face_data)
# Reshape the array based on the number of collected images
face_data = face_data.reshape(50, -1)

# Ensure the 'attendance' directory exists; create it if it doesn't
if not os.path.exists('attendance'):
    os.makedirs('attendance')

# Save or update the names and face data in the 'attendance' directory
if 'names.pkl' not in os.listdir('attendance/'):
    # If 'names.pkl' doesn't exist, create it and add the user's name repeated for all 50 samples
    names = [name] * 50
    with open('attendance/names.pkl', 'wb') as f:
        pickle.dump(names, f)
else:
    # If 'names.pkl' exists, load the existing names
    with open('attendance/names.pkl', 'rb') as f:
        names = pickle.load(f)
    # Add the new name repeated for all 50 samples
    names = names + [name] * 50
    # Save the updated names list
    with open('attendance/names.pkl', 'wb') as f:
        pickle.dump(names, f)

# Save or update the face data in the 'attendance' directory
if 'face_data.pkl' not in os.listdir('attendance/'):
    # If 'face_data.pkl' doesn't exist, create it and save the new face data
    with open('attendance/face_data.pkl', 'wb') as f:
        pickle.dump(face_data, f)
else:
    # If 'face_data.pkl' exists, load the existing face data
    with open('attendance/face_data.pkl', 'rb') as f:
        existing_data = pickle.load(f)
    # Stack the new face data with the existing data vertically using np.vstack
    faces = np.vstack((existing_data, face_data))
    # Save the updated face data
    with open('attendance/face_data.pkl', 'wb') as f:
        pickle.dump(faces, f)


In [4]:
import cv2
import numpy as np
import os
import pickle
import csv
import time
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime

# Initialize the webcam and face detector using the Haar Cascade Classifier
cap = cv2.VideoCapture(0)
face_detect = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load names and face data from previously saved pickle files
with open('attendance/names.pkl', 'rb') as w:
    LABELS = pickle.load(w)  # Names of people

with open('attendance/face_data.pkl', 'rb') as f:
    FACES = pickle.load(f)  # Corresponding face data

# Initialize the KNN classifier and train it with the loaded data
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(FACES, LABELS)

# Load a background image to overlay the webcam feed on
imagebackground = cv2.imread(r"C:\Users\ASUS\OneDrive\Desktop\Computer vision\attendance\blank-white-7sn5o1woonmklx1h.jpg")

# Column names for the CSV attendance file
COL_NAMES = ['NAME', 'TIME']

# Main loop for real-time face detection and attendance marking
while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Detect faces in the frame
    faces = face_detect.detectMultiScale(gray, scaleFactor=1.02, minNeighbors=5)

    # Loop over detected faces
    for (x, y, w, h) in faces:
        # Crop and resize the face for prediction
        crop_image = frame[y:y+h, x:x+w, :]
        resized_image = cv2.resize(crop_image, (50, 50)).flatten().reshape(1, -1)  # Reshape for the model
        # Predict the name of the person
        output = knn.predict(resized_image)

        # Get the current timestamp
        ts = time.time()
        date = datetime.fromtimestamp(ts).strftime("%d-%m-%Y")
        timestamp = datetime.fromtimestamp(ts).strftime("%H:%M:%S")

        # Create the CSV filename based on the current date
        csv_filename = 'attendance/Attendance_' + date + ".csv"
        exist = os.path.isfile(csv_filename)  # Check if the file already exists
        
        # Draw rectangles around the detected face and display the name
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        cv2.rectangle(frame, (x, y-40), (x+w, y), (50, 50, 255), -1)
        cv2.putText(frame, str(output[0]), (x, y-10), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 2)

        # Prepare the attendance record for the CSV file
        attendance = [str(output[0]), str(timestamp)]

        # Overlay the webcam frame onto the background image
        imagebackground[162:162+480, 55:55+640] = frame

    # Display the resulting frame with the background image
    cv2.imshow('frame', imagebackground)

    # Wait for key press; 'q' saves attendance, 'o' exits the loop
    k = cv2.waitKey(1)
    
    if k == ord('q'):
        time.sleep(5)  # Delay to ensure any processing is done before saving

        # Write to the CSV file
        if exist:
            # Append the new attendance record if the file already exists
            with open(csv_filename, 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(attendance)
        else:
            # Create the file with column headers and the first record if it doesn't exist
            with open(csv_filename, 'a', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(COL_NAMES)
                writer.writerow(attendance)
                
    if k == ord('o'):
        break  # Break the loop to stop the program

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
